<a href="https://colab.research.google.com/github/rastringer/code_first_ml/blob/main/matmul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pathlib import Path
import pickle, gzip, math, os, time, shutil, matplotlib as mpl, matplotlib.pyplot as plt

In [ ]:
import torch
from torch import tensor

### Let's download the MNIST dataset using PyTorch

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt


training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████| 9912422/9912422 [00:00<00:00, 104096203.14it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 8066570.81it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 49559229.77it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 20707096.49it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
len(training_data)

60000

In [ ]:
transform = transforms.ToTensor()

train_loader = torch.utils.data.DataLoader(training_data, batch_size=len(training_data), shuffle=True)
# Get full train set
x_train, y_train = next(iter(train_loader))

# Reshape to (num_samples, 784)
x_train = x_train.view(x_train.shape[0], -1)
x_train.shape

torch.Size([60000, 784])

We have 60,000 images of the numbers 0-9.
It would be helpful to access a single tensor however PyTorch's `DataLoader` divides the data into batches for efficient training.

We can access and view different image tensors by extracting images and labels from individual batches:

In [ ]:
from torch.utils.data import DataLoader


# Create a DataLoader to handle batching and shuffling
batch_size = 32
train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True)

# Access and view elements from the DataLoader
for batch in train_loader:
    images, labels = batch
    # You can now work with the batch of images and labels
    # For example, printing the shape of the batch
    print("Batch of images shape:", images.shape)
    print("Batch of labels shape:", labels.shape)
    image_tensor = images[:5]
    print("Image tensor shape:", image_tensor.shape)
    break  # Stop after processing the first batch

Batch of images shape: torch.Size([32, 1, 28, 28])
Batch of labels shape: torch.Size([32])
Image tensor shape: torch.Size([5, 1, 28, 28])


### What is shape?

The number of rows or columns in a tensor.

We also need to 'flatten' the image tensor since the current shape of [5, 1, 28, 28] (denoting 5 entries of 1 color channel and image size of 28 x 28 pixels) won't fit with our weights tensor.

In [ ]:
reshaped_images = image_tensor.view(image_tensor.size(0), -1)
reshaped = reshaped_images[:5]
reshaped.shape

torch.Size([5, 784])

In [ ]:
torch.manual_seed(1)
weights = torch.randn(784, 10)
bias = torch.zeros(10)

In [ ]:
a = reshaped
b = weights
a.shape, b.shape

(torch.Size([5, 784]), torch.Size([784, 10]))

In [ ]:
# a rows, a columns
ar, ac = a.shape
# b rows, b columns
br, bc = b.shape

(ar, ac), (br, bc)


((5, 784), (784, 10))

In [ ]:
t1 = torch.zeros(ar, bc)
t1.shape

torch.Size([5, 10])

In [ ]:
t1

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
def matmul_simple(a, b):
  (ar,ac),(br,bc) = a.shape,b.shape
  t1 = torch.zeros(ar, bc)
  for i in range(ar):
    for j in range(bc):
      for k in range(ac):
        t1[i][j] += a[i][k] * b[k][j]

  return t1

In [ ]:
%timeit matmul_simple(a, b)

956 ms ± 228 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
t1 = matmul_simple(a, b)
t1.shape

torch.Size([5, 10])

In [ ]:
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
t1

tensor([[  9.61,   4.74,  -8.53,  -5.38,  13.76,  -4.88, -19.45,  -8.67,   4.12,  19.27],
        [  0.33,  -1.38, -14.74,  -0.24, -11.11,  -4.17, -15.35,   1.32,  -6.79,   2.15],
        [  2.21,  -1.10, -12.73,  -0.47,   4.21,  -6.72, -15.29,  -3.26,  -4.43,  11.80],
        [ -0.71,   3.71,  -4.32,  -3.57,  12.49,  -6.21, -27.57, -16.23, -23.68,   3.00],
        [ -0.86,   1.47, -15.30,   2.81,  -3.78,  -0.57,  -7.47,   8.04, -22.77,  -5.03]])

### Python to machine code via **Numba**.
Numba means we can write Python that compiles, then runs at speed similar to C.

In [ ]:
import numba as nb
from numba import njit
import numpy as np

In [ ]:
a.shape

torch.Size([5, 784])

Numba doesn't work with PyTorch tensors, so we have to convert to numpy arrays

In [ ]:
a_np = a.numpy()
b_np = b.numpy()


In [ ]:
@nb.jit(nopython=True)
def matmul_numba(a, b):
  ar,ac = a_np.shape
  br,bc = b_np.shape
  t1 = np.zeros((ar, bc))
  for i in range(ar):
    for j in range(bc):
      dot_product = 0.0
      for k in range(ac):
        dot_product += a[i][k] * b[k][j]
      t1[i][j] = dot_product
  return t1

In [ ]:
%timeit matmul_numba(a_np, b_np)

62.7 µs ± 9.02 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Broadcasting

How can we perform efficient operation on objects of different shapes?

In [ ]:
a_tensor = tensor([10., 6, -4])
b_tensor = tensor([2., 8, 7])


In [ ]:
a_tensor + 1

tensor([11.,  7., -3.])

In [ ]:
c_tensor = tensor([10,20,30])
m_matrix = tensor([[1.,2.,3.],[4.,5.,6.],[7.,8.,9.]])

In [ ]:
m_matrix

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])

In [ ]:
m_matrix + c_tensor

tensor([[11., 22., 33.],
        [14., 25., 36.],
        [17., 28., 39.]])

In [ ]:
c_tensor[None,:].shape

torch.Size([1, 3])

How is this happening? Let's check using a little-known method, `expand_as`.

In [ ]:
c_tensor

tensor([10, 20, 30])

In [ ]:
expanded = c_tensor.expand_as(m_matrix)
expanded

tensor([[10, 20, 30],
        [10, 20, 30],
        [10, 20, 30]])

After the expansion, `expanded` now acts as if it is a 3 x 3 matrix (and is the correct shape to be multipled with m_matrix). Though it looks like it has copied itself three times, in memory it is still just three values in one row. It simply acts as if it's a 3 x 3 matrix.

In [ ]:
expanded.storage()

<ipython-input-30-fd5960d9720c>:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  expanded.storage()


 10
 20
 30
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 3]

### Strides

NumPy achieves this slight of hand as follows: when the `expanded` tensor uses `stride` to read through the three rows required (in this case), it is in reality going across its [10,20,30] values three times.

This is the also the technique used by deep learning frameworks such as JAX and PyTorch to avoid prohibitively expensive copies in order to multiply tensors and matrices of different shapes.

In [ ]:
expanded.stride(), expanded.shape

((0, 1), torch.Size([3, 3]))

### Unsqueeze and None

We can also tweak shapes using `unsqueeze` and `None`. In this case, from a tensor to a matrix of 1 x 3 columns.

In [ ]:
c_tensor

tensor([10, 20, 30])

In [ ]:
c_tensor.unsqueeze(0)

tensor([[10, 20, 30]])

`None` inserts a new axis, achieving the same result.

In [ ]:
# Create a matrix with one row
c_tensor[None,:]

tensor([[10, 20, 30]])

Unsqueezing into the first dimension means we have three rows of one column.

In [ ]:
c_tensor.unsqueeze(1)

tensor([[10],
        [20],
        [30]])

In [ ]:
# Create a matrix with one column
c_tensor[:, None]


tensor([[10],
        [20],
        [30]])

In [ ]:
t1

tensor([[  9.61,   4.74,  -8.53,  -5.38,  13.76,  -4.88, -19.45,  -8.67,   4.12,  19.27],
        [  0.33,  -1.38, -14.74,  -0.24, -11.11,  -4.17, -15.35,   1.32,  -6.79,   2.15],
        [  2.21,  -1.10, -12.73,  -0.47,   4.21,  -6.72, -15.29,  -3.26,  -4.43,  11.80],
        [ -0.71,   3.71,  -4.32,  -3.57,  12.49,  -6.21, -27.57, -16.23, -23.68,   3.00],
        [ -0.86,   1.47, -15.30,   2.81,  -3.78,  -0.57,  -7.47,   8.04, -22.77,  -5.03]])

2 * a matrix will be broadcast across all rows and columns

In [ ]:
2 * t1

tensor([[ 19.22,   9.48, -17.06, -10.76,  27.51,  -9.77, -38.91, -17.34,   8.24,  38.53],
        [  0.66,  -2.76, -29.49,  -0.49, -22.22,  -8.35, -30.70,   2.64, -13.59,   4.31],
        [  4.42,  -2.20, -25.45,  -0.94,   8.41, -13.43, -30.58,  -6.52,  -8.87,  23.60],
        [ -1.42,   7.42,  -8.64,  -7.15,  24.98, -12.42, -55.13, -32.45, -47.36,   6.00],
        [ -1.72,   2.94, -30.61,   5.62,  -7.56,  -1.14, -14.95,  16.08, -45.54, -10.07]])

Note that our original loops, which looked like this:
```
for i in range(ar):
    for j in range(bc):
      for k in range(ac):
        t1[i][j] += a[i][k] * b[k][j]
 ```
 have now been replaced by the single line `c[i] = ...`. This greatly speeds up the operation.

In [ ]:
def matmul_broadcast(a,b):
    (ar,ac),(br,bc) = a.shape,b.shape
    c = torch.zeros(ar, bc)
    for i in range(ar):
#       c[i,j] = (a[i,:] * b[:,j]).sum()      # previous version
        c[i]   = (a[i,:,None] * b).sum(dim=0) # broadcast version
    return c

In [ ]:
%timeit matmul_broadcast(a,b)

170 µs ± 31 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Einstein summation

_Einsum_ is a compact representation that combines sums and their products.

* Repeating charactes between input arrays mean values along those axis are multiplied together.

* Values along the axis of an omitted letter will be summed.

In [ ]:
a.shape, b.shape

(torch.Size([5, 784]), torch.Size([784, 10]))

In [ ]:
a

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [ ]:
b

tensor([[-1.53, -0.75, -0.65,  ..., -1.61, -0.71,  0.30],
        [-0.78, -0.25, -0.22,  ..., -1.16,  0.70,  0.20],
        [ 0.87,  0.24, -0.66,  ..., -1.45,  0.06, -0.62],
        ...,
        [ 0.51,  0.47, -0.26,  ...,  0.65,  0.43, -1.29],
        [ 0.52,  1.03,  0.81,  ..., -0.10,  2.26, -0.28],
        [-1.49,  0.39, -0.55,  ..., -0.19, -0.51,  0.54]])

In [ ]:
# This is the same as writing our variants
# from above
# c[i,j] = (a[i,:] * b[:,j]).sum()      # earlier version
# c[i]   = (a[i,:,None] * b).sum(dim=0) # broadcast version

matrix = torch.einsum('ik,kj->ikj', a, b)
matrix.shape

torch.Size([5, 784, 10])

In the result, we see `5, 784, 10`, which denotes:
_5_ for the original rows of _a_
_10_ for the original columns of _b_
and _784_ results the multiplication.

In [ ]:
# Results should match those for `t1` above.
matrix.sum(1)

tensor([[  9.61,   4.74,  -8.53,  -5.38,  13.76,  -4.88, -19.45,  -8.67,   4.12,  19.27],
        [  0.33,  -1.38, -14.74,  -0.24, -11.11,  -4.17, -15.35,   1.32,  -6.79,   2.15],
        [  2.21,  -1.10, -12.73,  -0.47,   4.21,  -6.72, -15.29,  -3.26,  -4.43,  11.80],
        [ -0.71,   3.71,  -4.32,  -3.57,  12.49,  -6.21, -27.57, -16.23, -23.68,   3.00],
        [ -0.86,   1.47, -15.30,   2.81,  -3.78,  -0.57,  -7.47,   8.04, -22.77,  -5.03]])

Let's look at what just happened in detail.
`'ik,kj->ikj'`
In this expression, the inputs are the the left of the `->`, the outputs to the right.
The characters denote rows and columns.
Here, we refer to two matrices, one of
`i` rows and `k` columns and one of
`k` rows and `j` columns.

The output is a new matrix containing
`i` dimensions and `j` columns.

The repeated index 'k' indicates that we sum over it. So for each output entry matrix[i,j], we compute:

matrix[i,j] = Σ_k a[i,k] * b[k,j]

In other words, we take the matrix product of a and b by summing over the common dimension k.

In this way, Enstein summation frees us from expliciting writing the multiplication in a loop. Broadcasting the matrices, multiplying elements and summing all happens in one line.

In [ ]:
# If we remove the `k` from the ouptut,
# the sum happens automatically
torch.einsum('ik,kj->ij', a, b)

tensor([[  9.61,   4.74,  -8.53,  -5.38,  13.76,  -4.88, -19.45,  -8.67,   4.12,  19.27],
        [  0.33,  -1.38, -14.74,  -0.24, -11.11,  -4.17, -15.35,   1.32,  -6.79,   2.15],
        [  2.21,  -1.10, -12.73,  -0.47,   4.21,  -6.72, -15.29,  -3.26,  -4.43,  11.80],
        [ -0.71,   3.71,  -4.32,  -3.57,  12.49,  -6.21, -27.57, -16.23, -23.68,   3.00],
        [ -0.86,   1.47, -15.30,   2.81,  -3.78,  -0.57,  -7.47,   8.04, -22.77,  -5.03]])

In [ ]:
def matmul_einsum(a, b):
  return torch.einsum('ik,kj->ij', a, b)

In [ ]:
%timeit matmul_einsum(x_train, weights)

31.9 ms ± 450 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


We can also just use `torch.matmul`

In [ ]:
%timeit torch.matmul(x_train, weights)

31.8 ms ± 890 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Cuda

GPUs vastly speed up operations by carrying out functions in parallel.

In [ ]:
def matmul(grid, a,b,c):
    i,j = grid
    if i < c.shape[0] and j < c.shape[1]:
        tmp = 0.
        for k in range(a.shape[1]): tmp += a[i, k] * b[k, j]
        c[i,j] = tmp

In [ ]:
res = torch.zeros(ar, bc)
matmul((0,0), a, b, res)
res

tensor([[9.61, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]])

In [ ]:
def launch_kernel(kernel, grid_x, grid_y, *args, **kwargs):
    for i in range(grid_x):
        for j in range(grid_y): kernel((i,j), *args, **kwargs)

In [ ]:
res = torch.zeros(ar, bc)
launch_kernel(matmul, ar, bc, a, b, res)
res

tensor([[  9.61,   4.74,  -8.53,  -5.38,  13.76,  -4.88, -19.45,  -8.67,   4.12,  19.27],
        [  0.33,  -1.38, -14.74,  -0.24, -11.11,  -4.17, -15.35,   1.32,  -6.79,   2.15],
        [  2.21,  -1.10, -12.73,  -0.47,   4.21,  -6.72, -15.29,  -3.26,  -4.43,  11.80],
        [ -0.71,   3.71,  -4.32,  -3.57,  12.49,  -6.21, -27.57, -16.23, -23.68,   3.00],
        [ -0.86,   1.47, -15.30,   2.81,  -3.78,  -0.57,  -7.47,   8.04, -22.77,  -5.03]])

In [ ]:
from numba import cuda

def matmul(grid, a,b,c):
    i,j = grid
    if i < c.shape[0] and j < c.shape[1]:
        tmp = 0.
        for k in range(a.shape[1]): tmp += a[i, k] * b[k, j]
        c[i,j] = tmp

In [ ]:
@cuda.jit
def matmul_cuda(a,b,c):
    i, j = cuda.grid(2)
    if i < c.shape[0] and j < c.shape[1]:
        tmp = 0.
        for k in range(a.shape[1]): tmp += a[i, k] * b[k, j]
        c[i,j] = tmp

In [ ]:
tr = matmul_broadcast(x_train, weights)

In [ ]:
r = np.zeros(tr.shape)
m1g,m2g,rg = map(cuda.to_device, (x_train,weights,r))

In [ ]:
r.shape

(60000, 10)

In [ ]:
TPB = 16
rr,rc = r.shape
blockspergrid = (math.ceil(rr / TPB), math.ceil(rc / TPB))
blockspergrid

(3750, 1)

In [ ]:
%%timeit -n 10
matmul_cuda[blockspergrid, (TPB,TPB)](m1g,m2g,rg)
r = rg.copy_to_host()

14.8 ms ± 9.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
m1c,m2c = x_train.cuda(),weights.cuda()

In [ ]:
r=(m1c@m2c).cpu()

In [ ]:
%timeit -n 10 r=(m1c@m2c).cpu()

3.68 ms ± 1.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
